In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-wav.tar

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import malaya_speech
malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.stream` is not able to use.


<module 'malaya_speech' from '/home/husein/dev/malaya-speech/malaya_speech/__init__.py'>

In [3]:
model = malaya_speech.speaker_vector.huggingface('microsoft/wavlm-base-plus-sv')

In [4]:
_ = model.cuda()

In [5]:
import json
import pickle
from tqdm import tqdm

In [6]:
with open('voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [7]:
sample_files[0]

'/home/husein/youtube/voxceleb-wav/aac-id08456-gWxUPq1QZlI-00257.wav'

In [8]:
with open('voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [9]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))

In [10]:
len(unique_files)

36237

In [11]:
import torch

In [12]:
vectors = {}
for f in tqdm(unique_files):
    y_, _ = malaya_speech.load(f.replace('/home/husein/youtube/', '/home/husein/'))
    try:
        v = model(y_)[0]
        vectors[f] = v
    except:
        torch.cuda.empty_cache()

  0%|                                                                                                          | 0/36237 [00:00<?, ?it/s]/home/husein/.local/lib/python3.8/site-packages/transformers/models/wavlm/modeling_wavlm.py:1755: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 36237/36237 [19:45<00:00, 30.58it/s]


In [13]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    if labels[i][1] in vectors and labels[i][2] in vectors:
        ls.append(labels[i][0])
        scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|██████████████████████████████████████████████████████████████████████████████████████| 5900000/5900000 [00:18<00:00, 320565.74it/s]


In [14]:
len(scores), len(vectors)

(5899593, 36234)

In [15]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [16]:
calculate_eer(ls, scores)

(0.06315593794598653, array(0.87164013))

In [18]:
v.shape

(512,)